In [1]:
print("RAG in Langchain")

RAG in Langchain


In [2]:
# Load Credentials
# secure way to load credentials
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
# setup LLM with groq
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# from langchain_groq import ChatGroq

# llm = ChatGroq(
#     model="llama-3.3-70b-versatile", 
#     temperature=0.2,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )

In [7]:
# data loaders -- used to load the data from a source
# pdf loader loads data from pdf

from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import WebBaseLoader

url = ["https://supercell.com/en/parents/"]

# for u in url:

web_loader = WebBaseLoader(url)
documents = web_loader.load()

# pdf_loader = PyMuPDFLoader("Data/SE-AI&DS-Syllabus.pdf")  # Replace with your PDF file path
# documents = pdf_loader.load()

In [8]:
print(documents)

[Document(metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's Guide × Supercell", 'description': '', 'language': 'en'}, page_content='Parent\'s Guide × Supercell\nGamesmo.coSquad BustersBrawl StarsClash RoyaleBoom BeachClash of ClansHay DayCareersWhy You Might Love It HereOur OfficesJoining SupercellLiving in HelsinkiLiving in ShanghaiOpen PositionsCareer StoriesSparkSupportBe Safe & Play FairAbout UsNewsInvestmentsLevel UpIlkka\'s Long TextsGames FirstGamesGamesmo.coSquad BustersBrawl StarsClash RoyaleBoom BeachClash of ClansHay DayCareersCareersWhy You Might Love It HereOur OfficesJoining SupercellLiving in HelsinkiLiving in ShanghaiOpen PositionsCareer StoriesSparkSupportSupportBe Safe & Play FairAbout UsAbout UsNewsInvestmentsLevel UpIlkka\'s Long TextsGames FirstParent\'s GuideBahasa IndonesiaBahasa MelayuDeutschEnglishEspañolFrançaisItalianoNederlandsPortuguêsSuomiTiếng ViệtTürkçeРусскийالعربيةفارسی한국어日本語简体中文繁體中文WHERE ARE SUPERCELL GAMES AVAILABLE?WHAT ARE

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [10]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings= GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000242C3308620>, model='models/text-embedding-004', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [11]:

from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(docs, embeddings)
vector_store

In [26]:
# Similarity Search
# searches in vector db and find the nodes with highest similarity scores

query = "who process in  app purchases?"

result = vector_store.similarity_search(query)
result

[Document(id='bd876298-d3c2-4e61-9058-1af6ef5db4d5', metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's Guide × Supercell", 'description': '', 'language': 'en'}, page_content='(like iPhone or iPad)Instructions for Google Android devices (like Samsung Galaxy, OnePlus, HTC, Nexus, etc)WHO PROCESSES IN-APP PURCHASES?Supercell does not process payments for in-app purchases, or have access to credit card information. Payment transactions themselves are processed through the App Store or Google Play (depending on your device) via the personal email address associated with that account. Transaction receipts are emailed to you by Apple or Google after each purchase.'),
 Document(id='77bb1885-cb2b-4b6f-8271-5705bdd505bf', metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's Guide × Supercell", 'description': '', 'language': 'en'}, page_content="process payments for in-app purchases, nor do we have access to any of your payment information. That's 

In [ ]:
# Multi Query Retriever
# Retrieves the most similar node from vector db and answer the question with the help of an Initiallized LLm

from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)

In [ ]:
# Make a chain to answer question and answer
from langchain.chains import RetrievalQA

# Create RAG Pipeline (Retrieval-Augmented Generation)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever_from_llm, 
    return_source_documents=True
)

In [23]:
query = "who process in  app purchases?"
response = qa_chain({"query": query})

print("Result")
print("-"*20)
print(response["result"])

print("Source")
print("-"*20)
print(response["source_documents"])

Result
--------------------
Payment transactions for in-app purchases are processed through the App Store or Google Play, depending on your device.
Source
--------------------
[Document(id='bd876298-d3c2-4e61-9058-1af6ef5db4d5', metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's Guide × Supercell", 'description': '', 'language': 'en'}, page_content='(like iPhone or iPad)Instructions for Google Android devices (like Samsung Galaxy, OnePlus, HTC, Nexus, etc)WHO PROCESSES IN-APP PURCHASES?Supercell does not process payments for in-app purchases, or have access to credit card information. Payment transactions themselves are processed through the App Store or Google Play (depending on your device) via the personal email address associated with that account. Transaction receipts are emailed to you by Apple or Google after each purchase.'), Document(id='77bb1885-cb2b-4b6f-8271-5705bdd505bf', metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's G

In [24]:
query = "Are my bank details private?"
response = qa_chain({"query": query})

print("Result")
print("-"*20)
print(response["result"])

print("Source")
print("-"*20)
print(response["source_documents"])

Result
--------------------
Supercell does not process payments for in-app purchases, or have access to your payment information. The payment transactions themselves are completed through Apple's App Store or Google Play (depending on your device) via your personal email address associated with that account.
Source
--------------------
[Document(id='11d58602-d6b6-405a-831e-3b159663b091', metadata={'source': 'https://supercell.com/en/parents/', 'title': "Parent's Guide × Supercell", 'description': '', 'language': 'en'}, page_content="with that account. Transaction receipts are emailed to you by Apple or Google after each purchase. Sometimes, purchases may be bundled together if you've bought multiple items, and it might take a few days for you to receive the receipt. These post-purchase processes are subject to change by Apple or Google. Supercell doesn't offer any billing programs.ARE MY BANK DETAILS KEPT PRIVATE?We don't process payments for in-app purchases, nor do we have access to 

In [25]:
query = "MY CHILD ACCIDENTALLY MADE AN IN-APP PURCHASE. CAN I GET A REFUND"
response = qa_chain({"query": query})


print("Result")
print("-"*20)
print(response["result"])

print("Source")
print("-"*20)
print(response["source_documents"])

Result
--------------------
As with most downloadable software products, items bought in our games (in-app purchases) are non-refundable. In rare cases, exceptions can be applied.

If the purchase was made on an Apple (iOS) device: For purchases made on an iOS device, Supercell is not able to handle refunds directly. Please visit Apple support and select the "Contact iTunes store support" link, then "Purchases, billing and redemption" and choose the option that fits you best.

If the purchase was made on a Google (Android) device: Please send an email to parents@supercell.com or contact us directly through in-game support by going to Settings > Help & Support and then tap "Contact Us" in the upper right-hand corner. In your message please state the name of the game the purchase was made in and add the receipt of the purchase. The transaction ID (looks similar to this - GPA.1234-1234-1234-12345) at the bottom of the document must be visible. Receipts for transactions have been emailed t